In [ ]:
#| default_exp asociacion

## Implementación del ejemplo con python

In [ ]:
#| echo: true

### Bibliotecas necesarias:

#! pip install mlxtend
#! pip install pandas

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Datos de transacciones: lista de listas, donde cada sublista representa una transacción
transactions = [
    ['Leche', 'Pan', 'Mantequilla'],
    ['Leche', 'Pan'],
    ['Leche', 'Manzana'],
    ['Pan', 'Mantequilla'],
    ['Leche', 'Pan', 'Mantequilla', 'Manzana'],
    ['Manzana', 'Mantequilla']
]

transactions

[['Leche', 'Pan', 'Mantequilla'],
 ['Leche', 'Pan'],
 ['Leche', 'Manzana'],
 ['Pan', 'Mantequilla'],
 ['Leche', 'Pan', 'Mantequilla', 'Manzana'],
 ['Manzana', 'Mantequilla']]

In [ ]:
#Creamos una instancia de TransactionEncoder. 
# Este objeto se utiliza para transformar nuestras transacciones en una matriz booleana.
te = TransactionEncoder()
te

TransactionEncoder()

## TransactionEncoder

**TransactionEncoder** 
- es una clase de la biblioteca mlxtend que se utiliza para convertir listas de transacciones en una matriz booleana, 
- lo que es especialmente útil para el análisis de asociaciones y la minería de datos.

**¿Qué es TransactionEncoder?**
- TransactionEncoder es un transformador que toma una lista de listas (donde cada sublista representa una transacción que contiene uno o más ítems)
- y la convierte en una matriz de formato booleano o de ceros y unos. 
- Cada fila de la matriz resultante representa una transacción, y cada columna representa un ítem. Los valores de la matriz indican la presencia (True o 1) o la ausencia (False o 0) de un ítem en cada transacción.

**¿Qué hace TransactionEncoder?**

**Ajuste y Transformación:**

**fit:** Ajusta el codificador a los datos, identificando todos los ítems únicos en el conjunto de transacciones.
    transform: Convierte las transacciones en una matriz booleana basada en los ítems identificados.
    fit_transform: Ajusta y transforma los datos en una sola operación.
    Matriz Booleana:
    La matriz booleana generada tiene las siguientes características:

**Filas:** Cada fila representa una transacción.
    Columnas: Cada columna representa un ítem único encontrado en las transacciones.
    Valores: True (o 1) indica que el ítem está presente en la transacción, y False (o 0) indica que el ítem no está presente.

In [ ]:
# Transformar los datos de transacciones en una matriz booleana
te_ary = te.fit_transform(transactions)
te_ary

array([[ True,  True, False,  True],
       [ True, False, False,  True],
       [ True, False,  True, False],
       [False,  True, False,  True],
       [ True,  True,  True,  True],
       [False,  True,  True, False]])

In [ ]:
# Convertir la matriz booleana en un DataFrame de pandas
# te_ary: es la matriz booleana generada por TransactionEncoder, donde cada fila representa una transacción
# y cada columna representa un producto. Un valor 1 indica que el producto está presente en la transacción,
# y un valor 0 indica que no lo está.
# columns=te.columns_: son los nombres de las columnas (los nombres de los productos) que se asignan a las columnas del DataFrame
df = pd.DataFrame(te_ary, columns=te.columns_)

# Mostrar el DataFrame resultante para verificar la transformación
df


,Leche,Mantequilla,Manzana,Pan
0,True,True,False,True
1,True,False,False,True
2,True,False,True,False
3,False,True,False,True
4,True,True,True,True
5,False,True,True,False


In [ ]:
# Calcular los ítems frecuentes utilizando el algoritmo Apriori
# Utilizamos la función apriori para encontrar ítems frecuentes en las transacciones
# - df: El DataFrame que contiene las transacciones transformadas
# - min_support=0.1: Definimos el soporte mínimo como 0.1 (10%). Esto significa que un ítem debe aparecer en al menos el 10% de las transacciones para ser considerado frecuente.
# - use_colnames=True: Usamos los nombres de las columnas (los nombres de los productos) en lugar de índices numéricos
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)

# Mostrar los ítems frecuentes y su soporte
print("Ítems frecuentes encontrados:")
frequent_itemsets


Ítems frecuentes encontrados:


,support,itemsets
0,0.666667,(Leche)
1,0.666667,(Mantequilla)
2,0.500000,(Manzana)
3,0.666667,(Pan)
4,0.333333,"(Leche, Mantequilla)"
5,0.333333,"(Manzana, Leche)"
6,0.500000,"(Pan, Leche)"
7,0.333333,"(Manzana, Mantequilla)"
8,0.500000,"(Pan, Mantequilla)"
9,0.166667,"(Pan, Manzana)"


In [ ]:
# Generar las reglas de asociación
# Utilizamos la función association_rules para generar las reglas a partir de los ítems frecuentes.
# - frequent_itemsets: contiene los ítems frecuentes calculados con el algoritmo Apriori.
# - metric: definimos la métrica "confidence" para evaluar las reglas.
# - min_threshold: establecemos un umbral mínimo de 0.5 (50%) para la métrica de confianza.
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Filtrar y mostrar las reglas
# Seleccionamos las columnas más relevantes para mostrar: antecedentes, consecuentes, soporte, confianza y lift.
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
rules


Reglas de asociación generadas:


,antecedents,consequents,support,confidence,lift
0,(Leche),(Mantequilla),0.333333,0.500000,0.750
1,(Mantequilla),(Leche),0.333333,0.500000,0.750
2,(Manzana),(Leche),0.333333,0.666667,1.000
3,(Leche),(Manzana),0.333333,0.500000,1.000
4,(Pan),(Leche),0.500000,0.750000,1.125
5,(Leche),(Pan),0.500000,0.750000,1.125
6,(Manzana),(Mantequilla),0.333333,0.666667,1.000
7,(Mantequilla),(Manzana),0.333333,0.500000,1.000
8,(Pan),(Mantequilla),0.500000,0.750000,1.125
9,(Mantequilla),(Pan),0.500000,0.750000,1.125


In [ ]:
# Filtrar y mostrar las reglas
# Seleccionamos solo las columnas relevantes para mostrar
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
rules



Reglas de asociación generadas:


,antecedents,consequents,support,confidence,lift
0,(Leche),(Mantequilla),0.333333,0.500000,0.750
1,(Mantequilla),(Leche),0.333333,0.500000,0.750
2,(Manzana),(Leche),0.333333,0.666667,1.000
3,(Leche),(Manzana),0.333333,0.500000,1.000
4,(Pan),(Leche),0.500000,0.750000,1.125
5,(Leche),(Pan),0.500000,0.750000,1.125
6,(Manzana),(Mantequilla),0.333333,0.666667,1.000
7,(Mantequilla),(Manzana),0.333333,0.500000,1.000
8,(Pan),(Mantequilla),0.500000,0.750000,1.125
9,(Mantequilla),(Pan),0.500000,0.750000,1.125


In [ ]:
# Interpretación del resultado
# Iterar sobre cada regla para imprimir una interpretación más clara
for idx, rule in rules.iterrows():
    # Convertir los conjuntos de antecedentes y consecuentes en cadenas de texto
    antecedents = ', '.join(list(rule['antecedents']))
    consequents = ', '.join(list(rule['consequents']))
    support = rule['support']
    confidence = rule['confidence']
    lift = rule['lift']
    
    # Imprimir la interpretación de cada regla
    print(f"Regla: {antecedents} -> {consequents}")
    print(f"  Soporte: {support:.2f}")
    print(f"  Confianza: {confidence:.2f}")
    print(f"  Lift: {lift:.2f}")
    print(f"  Interpretación: Si un cliente compra {antecedents}, hay una confianza del {confidence:.2%} de que también comprará {consequents}. El lift de {lift:.2f} indica que esta relación es {lift:.2f} veces más probable que si los productos fueran independientes.\n")


Regla: Leche -> Mantequilla
  Soporte: 0.33
  Confianza: 0.50
  Lift: 0.75
  Interpretación: Si un cliente compra Leche, hay una confianza del 50.00% de que también comprará Mantequilla. El lift de 0.75 indica que esta relación es 0.75 veces más probable que si los productos fueran independientes.

Regla: Mantequilla -> Leche
  Soporte: 0.33
  Confianza: 0.50
  Lift: 0.75
  Interpretación: Si un cliente compra Mantequilla, hay una confianza del 50.00% de que también comprará Leche. El lift de 0.75 indica que esta relación es 0.75 veces más probable que si los productos fueran independientes.

Regla: Manzana -> Leche
  Soporte: 0.33
  Confianza: 0.67
  Lift: 1.00
  Interpretación: Si un cliente compra Manzana, hay una confianza del 66.67% de que también comprará Leche. El lift de 1.00 indica que esta relación es 1.00 veces más probable que si los productos fueran independientes.

Regla: Leche -> Manzana
  Soporte: 0.33
  Confianza: 0.50
  Lift: 1.00
  Interpretación: Si un cliente compr

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()